# Supabase 사용 가이드

Supabase를 사용하는 방법을 설명합니다.

## 1. 환경 설정

### 1.1 필요한 패키지 설치

```bash
uv add supabase python-dotenv
```

## 2 환경 변수 설정

프로젝트 루트에 `.env` 파일을 생성하고 다음 내용을 추가합니다:

```
SUPABASE_URL=https://your-project-id.supabase.co
SUPABASE_KEY=your-anon-key
```

### Supabase 키 확인 방법

1. [Supabase 대시보드](https://supabase.com/dashboard)에 로그인
2. 프로젝트 선택
3. **Settings** > **API** 메뉴로 이동
4. **Project URL**과 **anon public** 키를 복사

![Supabase API 설정 위치](https://supabase.com/docs/img/guides/api/api-settings.png)

> **주의**: `service_role` 키는 절대 클라이언트에 노출하지 마세요. 일반적인 데이터 조회에는 `anon` 키를 사용합니다.

## 3. Supabase 연결하기

### 3.1 직접 연결 (기본 방법)

In [ ]:
import os
from dotenv import load_dotenv
from supabase import create_client, Client

# .env 파일에서 환경 변수 로드
load_dotenv()

# Supabase 클라이언트 생성
url = os.environ.get("SUPABASE_URL")
key = os.environ.get("SUPABASE_KEY")

client: Client = create_client(url, key)

print("Supabase 클라이언트 연결 성공!")

### 3.2 프로젝트 유틸리티 사용 (권장)

이 프로젝트에서는 `src/database` 모듈에 미리 정의된 함수들을 제공합니다.

In [ ]:
import sys
import os

# 프로젝트 루트를 경로에 추가
sys.path.append(os.path.abspath('..'))

from src.database.supabase_client import get_supabase_client

# 싱글톤 패턴으로 클라이언트 가져오기 (캐싱됨)
client = get_supabase_client()

print("프로젝트 클라이언트 연결 성공!")

## 4. 데이터 조회하기 (SELECT)

### 4.1 전체 데이터 조회

In [ ]:
# 테이블의 모든 데이터 조회
response = client.table('products').select('*').execute()

print(f"조회된 데이터 수: {len(response.data)}개")
print(f"첫 번째 레코드: {response.data[0] if response.data else 'None'}")

### 4.2 특정 컬럼만 조회

In [ ]:
# 필요한 컬럼만 선택하여 조회
response = client.table('products').select('name, price, product_rating').execute()

print(f"조회된 데이터: {response.data[:3]}")

## 5. Pandas DataFrame으로 변환하기

데이터 분석 시 Pandas DataFrame으로 변환하면 편리합니다.

In [ ]:
import pandas as pd

# 방법 1: 직접 변환
response = client.table('products').select('*').execute()
products_df = pd.DataFrame(response.data)

print(f"DataFrame 크기: {products_df.shape}")
products_df.head(3)

In [ ]:
# 방법 2: 프로젝트 유틸리티 함수 사용 (권장)
from src.database import load_table

products_df = load_table('products')
sellers_df = load_table('sellers')
reviews_df = load_table('reviews')
questions_df = load_table('questions')

print(f"상품 데이터: {products_df.shape}")
print(f"판매자 데이터: {sellers_df.shape}")
print(f"리뷰 데이터: {reviews_df.shape}")
print(f"질문 데이터: {questions_df.shape}")

## 6. 데이터 삽입 (INSERT)

> **참고**: 읽기 전용 권한만 있는 경우 아래 코드는 실행되지 않습니다.

In [ ]:
# 단일 레코드 삽입
# new_data = {
#     'name': '테스트 상품',
#     'price': 10000,
#     'product_rating': 4.5
# }
# response = client.table('products').insert(new_data).execute()

# 여러 레코드 삽입
# new_data_list = [
#     {'name': '상품1', 'price': 10000},
#     {'name': '상품2', 'price': 20000}
# ]
# response = client.table('products').insert(new_data_list).execute()

print("INSERT 예시 코드입니다 (주석 처리됨)")

## 7. 데이터 수정 (UPDATE)

In [ ]:
# 특정 조건의 레코드 수정
# response = (
#     client.table('products')
#     .update({'price': 15000})
#     .eq('id', 1)
#     .execute()
# )

print("UPDATE 예시 코드입니다 (주석 처리됨)")

## 8. 데이터 삭제 (DELETE)

In [ ]:
# 특정 조건의 레코드 삭제
# response = (
#     client.table('products')
#     .delete()
#     .eq('id', 1)
#     .execute()
# )

print("DELETE 예시 코드입니다 (주석 처리됨)")

## 9. 에러 처리

Supabase 쿼리 실행 시 발생할 수 있는 에러를 처리하는 방법입니다.

In [ ]:
from postgrest.exceptions import APIError

try:
    response = client.table('products').select('*').execute()
    print(f"성공: {len(response.data)}개 조회됨")
except APIError as e:
    print(f"API 에러: {e.message}")
except Exception as e:
    print(f"알 수 없는 에러: {e}")

## 10. 프로젝트 테이블 구조

이 프로젝트에서 사용하는 테이블 목록입니다.

| 테이블명 | 설명 | 주요 컬럼 |
|----------|------|----------|
| `products` | 상품 정보 | name, price, product_rating, category |
| `sellers` | 판매자 정보 | satisfaction_score, is_abusing_seller |
| `reviews` | 상품 리뷰 | review_text, review_rating |
| `questions` | 상품 문의 | question, answer |

In [ ]:
# 각 테이블의 컬럼 정보 확인
from src.database import load_table

tables = ['products', 'sellers', 'reviews', 'questions']

for table_name in tables:
    df = load_table(table_name)
    print(f"\n=== {table_name} 테이블 ===")
    print(f"크기: {df.shape[0]}행 x {df.shape[1]}열")
    print(f"컬럼: {', '.join(df.columns.tolist())}")

## 11. 참고 자료

- [Supabase 공식 문서](https://supabase.com/docs)
- [supabase-py GitHub](https://github.com/supabase-community/supabase-py)
- [PostgREST 필터 문서](https://postgrest.org/en/stable/api.html#operators)